<a href="https://colab.research.google.com/github/rozaxa/Artificial-Intelligence-Workshop-II/blob/Optuna/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [18]:
!pip install --quiet optuna

In [19]:
import optuna

optuna.__version__

'4.0.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [20]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection


def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.

    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=5, max_depth=3)  # Define the model.

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()  # Train and evaluate the model.


print("Accuracy: {}".format(objective()))

Accuracy: 0.96


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [21]:
import optuna


def objective(trial):
    iris = sklearn.datasets.load_iris()

    n_estimators = trial.suggest_int("n_estimators", 2, 20)
    max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))

    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-10-31 19:20:17,083] A new study created in memory with name: no-name-829fd186-7f26-4361-a0c9-43362f8ad28d
[I 2024-10-31 19:20:17,139] Trial 0 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 2, 'max_depth': 4.683848580290378}. Best is trial 0 with value: 0.9466666666666667.
[I 2024-10-31 19:20:17,293] Trial 1 finished with value: 0.9733333333333333 and parameters: {'n_estimators': 19, 'max_depth': 7.3588896708355}. Best is trial 1 with value: 0.9733333333333333.
[I 2024-10-31 19:20:17,395] Trial 2 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 10, 'max_depth': 2.2095017834472204}. Best is trial 1 with value: 0.9733333333333333.
[I 2024-10-31 19:20:17,437] Trial 3 finished with value: 0.94 and parameters: {'n_estimators': 4, 'max_depth': 14.533238133936813}. Best is trial 1 with value: 0.9733333333333333.
[I 2024-10-31 19:20:17,509] Trial 4 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 13, 'max_depth': 

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 19, 'max_depth': 7.3588896708355}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [22]:
import sklearn.svm


def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical("classifier", ["RandomForest", "SVC"])

    if classifier == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 2, 20)
        max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth
        )
    else:
        c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)

        clf = sklearn.svm.SVC(C=c, gamma="auto")

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-10-31 19:20:35,773] A new study created in memory with name: no-name-39e43a03-9ecc-405f-a1b5-d78d8c6ba1e9
[I 2024-10-31 19:20:35,821] Trial 0 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 0.00031242958577736}. Best is trial 0 with value: 0.32.
[I 2024-10-31 19:20:35,953] Trial 1 finished with value: 0.9266666666666666 and parameters: {'classifier': 'RandomForest', 'n_estimators': 13, 'max_depth': 1.7077557240357757}. Best is trial 1 with value: 0.9266666666666666.
[I 2024-10-31 19:20:36,041] Trial 2 finished with value: 0.94 and parameters: {'classifier': 'RandomForest', 'n_estimators': 2, 'max_depth': 6.081074004510159}. Best is trial 2 with value: 0.94.
[I 2024-10-31 19:20:36,100] Trial 3 finished with value: 0.9466666666666667 and parameters: {'classifier': 'SVC', 'svc_c': 460.27580601779846}. Best is trial 3 with value: 0.9466666666666667.
[I 2024-10-31 19:20:36,313] Trial 4 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 

Accuracy: 0.9733333333333333
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 9, 'max_depth': 13.694960234176065}


### Plotting the study

Plotting the optimization history of the study.

In [23]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [24]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [25]:
optuna.visualization.plot_contour(study, params=["n_estimators", "max_depth"])